In [2]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv('/content/dataset.csv')

In [ ]:
df.head(18)

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
5,5,BHMBCCMKT01,577,26.144536,91.736172,177,car,low,3,0,04-10-2016,10:26:00
6,6,BHMBCCMKT01,577,26.144536,91.736172,219,truck,high,6,0,04-10-2016,10:59:00
7,7,BHMBCCMKT01,577,26.144536,91.736172,247,car,average,5,0,04-10-2016,11:25:00
8,8,BHMBCCMKT01,577,26.144536,91.736172,259,cycle,average,5,0,04-10-2016,11:59:00
9,9,BHMBCCMKT01,577,26.144536,91.736172,266,bike,high,8,0,04-10-2016,12:29:00


In [ ]:
df.columns

Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime'],
      dtype='object')

In [ ]:
df.shape

(18368, 12)

In [ ]:
df.isnull().sum()

,0
ID,0
SystemCodeNumber,0
Capacity,0
Latitude,0
Longitude,0
Occupancy,0
VehicleType,0
TrafficConditionNearby,0
QueueLength,0
IsSpecialDay,0


In [ ]:
df.isnull().sum().sum()

np.int64(0)

In [ ]:
df.dtypes

,0
ID,int64
SystemCodeNumber,object
Capacity,int64
Latitude,float64
Longitude,float64
Occupancy,int64
VehicleType,object
TrafficConditionNearby,object
QueueLength,int64
IsSpecialDay,int64


###Model 1

In [ ]:
def update_price(previous_price, occupancy,perv_occupancy, capacity, alpha):
      occupancy_chnage= occupancy-perv_occupancy
      capacity_chnage= capacity-perv_occupancy
      occupancy_ratio = occupancy_chnage / capacity_chnage if capacity_chnage != 0 else 0
      new_price = previous_price + alpha * occupancy_ratio
      return new_price

In [ ]:
def simulate_dynamic_pricing(df, starting_price, alpha):
    df['UpdatedPrice'] = starting_price

    for pid in df['ID'].unique():
        spot_df = df[df['ID'] == pid]
        for date in spot_df['LastUpdatedDate'].unique():
            current_price = starting_price
            mask = (df['ID'] == pid) & (df['LastUpdatedDate'] == date)
            prev_ocuppancy=0;
            for idx in df[mask].index:
                row = df.loc[idx]
                current_price = update_price(current_price, row['Occupancy'], prev_ocuppancy, row['Capacity'], alpha)
                prev_ocuppancy=row['Occupancy']
                df.loc[idx, 'UpdatedPrice'] = current_price

    return df

In [ ]:
def find_best_alpha(df, starting_price=10, alpha_range=np.linspace(0.2, 0.5, 2)):
    best_alpha = None
    lowest_std = float('inf')
    temp_df = df.copy()

    for alpha in alpha_range:
        result_df = simulate_dynamic_pricing(temp_df, starting_price, alpha)

        result_df['PriceChange'] = result_df.groupby(['SystemCodeNumber', 'LastUpdatedDate'])['UpdatedPrice'].diff().fillna(0)

        day_wise_std = result_df.groupby(['SystemCodeNumber', 'LastUpdatedDate'])['PriceChange'].std()

        # Take mean of those std deviations
        mean_std_dev = day_wise_std.mean()

        if mean_std_dev < lowest_std:
            lowest_std = mean_std_dev
            best_alpha = alpha

    return best_alpha, lowest_std

In [ ]:
best_alpha, lowest_std=find_best_alpha(df)

/tmp/ipython-input-77-388538432.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.021143847487002' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[idx, 'UpdatedPrice'] = current_price
/tmp/ipython-input-77-388538432.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.052859618717504' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[idx, 'UpdatedPrice'] = current_price


In [ ]:
best_alpha

###Model 2

In [13]:
import pandas as pd
import numpy as np
from itertools import product

def simulate_price(df, base_price, alpha, beta, gamma, delta, epsilon, lambd=0.6):
    df = df.copy()

      # Map VehicleType to numeric weights
    vehicle_type_map = {'Car': 1, 'Bike': 0.7, 'Truck': 1.3}
    df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_type_map).fillna(1).astype(float)

    # Map TrafficConditionNearby to numeric values
    traffic_map = {'Low': 0.3, 'Medium': 0.6, 'High': 1.0}
    df['TrafficConditionNearby'] = df['TrafficConditionNearby'].map(traffic_map).fillna(0).astype(float)



    df['RawDemand'] = (
        alpha * (df['Occupancy'] / df['Capacity']) +
        beta * df['QueueLength'] -
        gamma * df['TrafficConditionNearby'] +
        delta * df['IsSpecialDay'] +
        epsilon * df['VehicleTypeWeight']
    )

    # Day-wise normalization
    df['NormalizedDemand'] = df.groupby(['LastUpdatedDate'])['RawDemand'].transform(
        lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0
    )

    df['UpdatedPrice'] = base_price * (1 + lambd * df['NormalizedDemand'])
    df['UpdatedPrice'] = df['UpdatedPrice'].clip(lower=0.5 * base_price, upper=2 * base_price)

    return df


In [14]:

def find_best_constants(df, base_price=10, lambd=0.6, steps=1):
    # Define ranges to search over
    alpha_range = np.linspace(0.1, 1.0, steps)
    beta_range  = np.linspace(0.1, 1.0, steps)
    gamma_range = np.linspace(0.1, 1.0, steps)
    delta_range = np.linspace(0.1, 1.0, steps)
    epsilon_range = np.linspace(0.1, 1.0, steps)

    best_constants = None
    lowest_mean_std = float('inf')

    for alpha, beta, gamma, delta, epsilon in product(alpha_range, beta_range, gamma_range, delta_range, epsilon_range):
        result_df = simulate_price(df, base_price, alpha, beta, gamma, delta, epsilon, lambd)
        result_df['PriceChange'] = result_df.groupby(['SystemCodeNumber', 'LastUpdatedDate'])['UpdatedPrice'].diff().fillna(0)

        day_wise_std = result_df.groupby(['SystemCodeNumber', 'LastUpdatedDate'])['PriceChange'].std()
        mean_std = day_wise_std.mean()

        if mean_std < lowest_mean_std:
            lowest_mean_std = mean_std
            best_constants = (alpha, beta, gamma, delta, epsilon)

    return best_constants, lowest_mean_std

In [15]:
best_constants, best_score = find_best_constants(df)
print(f"Optimal constants (α, β, γ, δ, ε): {best_constants}")
print(f"Lowest mean day-wise std: {best_score:.4f}")

Optimal constants (α, β, γ, δ, ε): (np.float64(0.1), np.float64(0.1), np.float64(0.1), np.float64(0.1), np.float64(0.1))
Lowest mean day-wise std: 0.8725
